In [ ]:
import rmgpy
from rmgpy.data.rmg import RMGDatabase
from rmgpy.rmg.react import *
from rmgpy.reaction import Reaction
from rmgpy.molecule.molecule import Molecule
from rmgpy.molecule.resonance import *
from rmgpy.species import Species

from base64 import b64encode
from IPython.display import display, HTML, Image

In [ ]:
families = ['Intra_R_Add_Endocyclic', 'Intra_R_Add_Exocyclic', 'Intra_R_Add_Polycyclic']

In [ ]:
databasePath = rmgpy.settings['database.directory']

database = RMGDatabase()
database.load(
    path = databasePath,
    thermoLibraries = [],
    reactionLibraries = [],
    seedMechanisms = [],
    kineticsFamilies = families,
    )

In [ ]:
spc = Species().fromSMILES('C1=CC=CC=C1C[CH2]')
spc.generateResonanceIsomers()
display(spc)

reactants = (spc,)

In [ ]:
# html settings
full = 24
half = full / 2
quarter = full / 4
eighth = full / 8

combos = getMoleculeTuples(reactants)

reactionList = []
for combo in combos:
    reactionList.extend(reactMolecules(combo))

reactionList = findDegeneracies(reactionList)
correctDegeneracyOfReverseReactions(reactionList, list(reactants))
reduceSameReactantDegeneracy(reactionList)

for reaction in reactionList:
    template = database.kinetics.families[reaction.family].retrieveTemplate(reaction.template)

    html = ['<table style="width:100%;table-layout:fixed;"><tr>']
    html += ['<td colspan="{0}" rowspan="3"><img style="display: block; margin: auto;" src="data:image/png;base64,{1}"></td>'.format(3*eighth, b64encode(reaction._repr_png_()))]
    html += ['<th colspan="{0}">Family</th>'.format(eighth)]
    html += ['<td colspan="{0}">{1}</td>'.format(half, reaction.family)]
    html += ['</tr><tr>']
    html += ['<th colspan="{0}">Template</th>'.format(eighth)]
    html += ['<td colspan="{0}">{1}</td>'.format(half, reaction.template)]
    html += ['</tr><tr>']
    html += ['<th colspan="{0}">Degeneracy</th>'.format(eighth)]
    html += ['<td colspan="{0}">{1}</td>'.format(half, reaction.degeneracy)]
    html += ['</tr><tr>']
    for entry in template:
        html += ['<td colspan="{0}" style="text-align: center;">{1}</td>'.format(full/len(template), entry.label)]
    html += ['</tr><tr>']
    for entry in template:
        html += ['<td colspan="{0}"><img style="display: block; margin: auto;" src="data:image/png;base64,{1}"></td>'.format(full/len(template), b64encode(entry.item._repr_png_()))]
    html += ['</tr></table>']

    display(HTML(''.join(html)))